In [1]:
from pyspark import SparkContext,SparkConf
import numpy as np
#import ujson
import pickle
import time

In [2]:
def safe_parse(raw_json):
    try:
        json_object = ujson.loads(raw_json)
        if 'user' not in json_object.keys():
            return False
        pass
    except ValueError, e:
        return False
    return True

def create_tuple(raw_json):
    json_object = ujson.loads(raw_json)
    return (json_object['user']['id_str'],json_object['text'])

In [5]:
partition = pickle.load(open("../../Data/users-partition.pickle", "r"))

def create_group_tuple(x):
    try: 
        gid=partition[x]
        return (gid,1)
    except:
        return (7,1)

In [6]:
conf = SparkConf()
conf.setMaster('local[2]')
conf.setAppName('tuning-test')
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[2]) created by <module> at /usr/local/lib/python2.7/dist-packages/IPython/utils/py3compat.py:288 

In [8]:
print "Kryo Serializer"

DataRDD=sc.textFile(','.join([line[:-1] for line in open('../../Data/hw2-files.txt')]))

st=time.time()

UserTextRDD = DataRDD.map(lambda x: x.encode('utf-8')).filter(lambda x: safe_parse(x))\
                .map(lambda x: create_tuple(x))
GroupCountRDD=UserTextRDD.map(lambda x: create_group_tuple(x[0])).reduceByKey(lambda x,y: x+y).sortByKey().cache()

print time.time()-st

Kryo Serializer


Py4JJavaError: An error occurred while calling o22.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/vagrant/UCSD_BigData_2016/Notebooks/Data/hw2-input.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:207)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)

In [6]:
sc.stop()

In [7]:
sc = SparkContext('local[2]', 'tuning-test2')

In [8]:
print "Java Serializer"

DataRDD=sc.textFile(','.join([line[:-1] for line in open('./Data/hw2-files.txt')]))

st=time.time()

UserTextRDD = DataRDD.map(lambda x: x.encode('utf-8')).filter(lambda x: safe_parse(x))\
                .map(lambda x: create_tuple(x))
GroupCountRDD=UserTextRDD.map(lambda x: create_group_tuple(x[0])).reduceByKey(lambda x,y: x+y).sortByKey().cache()

print time.time()-st

Java Serializer
7.79762101173


In [9]:
sc.stop()